<a href="https://colab.research.google.com/github/chanhyeong00/machine_learning_study/blob/main/Pytorch/basic_%EB%AA%A8%EB%8D%B8_%EC%A0%80%EC%9E%A5_%EB%B0%8F_%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

파이토치의 모델은 직렬화(Serialize)와 역직렬화(Deserialize)를 통해 객체를 저장하고 불러올 수 있다.

- 모델을 저장하려면 피클(Pickle)을 활용해 파이썬 객체 구조를 바이너리 프로토콜(Binary Protocols)로 직렬화한다. 모델에 사용된 텐서나 매개변수 저장

- 모델을 불러오려면 저장된 객체 파일을 역직렬화해 현재 프로세스의 메모리에 업로드한다. 이를 통해 모델을 통해 계산된 텐서나 매개변수를 불러올 수 있다.

모델 파일 확장자는 주로 .pt 나 .pth로 저장.(텐서플로는 .tflite)

### 모델 전체 저장/불러오기

이거도 데이터셋 없어서 코드만

In [1]:
import torch
from torch import nn

class CustomModel(nn.Module):
  def __init__(self):
    super.__init__()
    self.layer = nn.Linear(2, 1)

  def forward(self, x):
    x = self.layer(x)
    return x

앞의 장에서 모델을 만들어 훈련했다고 생각하고 코드를 본다.

In [ ]:
# 모델 저장
torch.save(
    model,
    "../models/model.pt"
)
torch.save(
    model.state_dict(),
    "../models/model_state_dict.pt"
) # 특정 시점 모델 상태

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load("../models/model.pt", map_location=device) # 모델을 불러온다.
print(model)

In [ ]:
with torch.no_grad(): # 불러온 모델 평가(no_grad는 기울기 계산을 하지 않음, 메모리 아낌)
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

### 모델 구조 확인

In [ ]:
import torch
from torch import nn


class CustomModel(nn.Module):
    pass
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load("../models/model.pt", map_location=device)
print(model)

### 모델 상태 저장/불러오기

In [ ]:
torch.save(
    model.state_dict(),
    "../models/model_state_dict.pt"
)

모델상태(torch.stacte_dict)는 모델에서 학습이 가능한 매개변수를 순서가 있는 딕셔너리 형식으로 반환한다.

OrderDict([

- layer.weight, tensor([[..]]), device='cuda:0'
- layer.bias, tensor([[..]]), device='cuda:0'

])

### 모델 상태 불러오기

모델 구조가 같아야 한다.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)

# torch.load로 상태 불러옴
model_state_dict = torch.load("../models/model_state_dict.pt", map_location=device)
# 모델 상태만 불러온 상태이므로 model.load_state_dict로 모델에 모델 상태 반영
model.load_state_dict(model_state_dict)

### 체크포인트 저장

체크포인트(Checkpoint)는 학습 과정의 특정 지점마다 저장하는 것을 의미.

학습과정에서 한 번에 전체 에폭을 반복하기 어렵거나 모종의 이유로 학습이 중단될 수 있다.

이러한 현상을 방지하기 위해 일정 에폭마다 학습된 결과를 저장해 나중에 이어서 학습하게 할 수 있다.

In [ ]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x
train_dataset = CustomDataset("../datasets/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

**여기가 체크포인트 저장하는 코드**

In [ ]:
checkpoint = 1
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    # 1000에폭마다 저장
    if (epoch + 1) % 1000 == 0:
        torch.save(
            {
                "model": "CustomModel",
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "cost": cost,
                "description": f"CustomModel 체크포인트-{checkpoint}",
            },
            f"../models/checkpoint-{checkpoint}.pt",
        )
        checkpoint += 1

다양한 정보를 저장하기 위해서 딕셔너리 형식으로 값을 할당한다.

- 학습을 이어서 진행하기 위한 목적이므로 epochs, 모델 상태(model.state_dict), 최적화상태(optimizer.state_dict) 등은 필수로 포함돼야 한다.

### 체크포인트 불러오기


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device) # 모델 장치에 올리고
criterion = nn.MSELoss().to(device) # 손실함수도 올리고
optimizer = optim.SGD(model.parameters(), lr=0.0001) # 옵티마이저 생성
checkpoint = torch.load("../models/checkpoint-6.pt") # 체크포인트 불러오기
model.load_state_dict(checkpoint["model_state_dict"]) # 모델에 그 체크포인트 상태 반영
optimizer.load_state_dict(checkpoint["optimizer_state_dict"]) # 옵티마이저에 상태 반영
checkpoint_epoch = checkpoint["epoch"] # 몇 에폭인지 불러옴
checkpoint_description = checkpoint["description"] # 몇 번쨰 체크포인트인지


print(checkpoint_description) # 몇 번쨰 체크포인트인지
# 이 모델로 훈련
for epoch in range(checkpoint_epoch + 1, 10000): # 체크포인트 에폭 이후부터 훈련
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss
        if (epoch + 1) % 1000 == 0:
            print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")